In [4]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

## Index Migration (v2 to v3)

This notebook is used to maintain data model parity with older indexes for version 3.0 of GraphRAG. If you have a pre-3.0 index and need to migrate without re-running the entire pipeline, you can use this notebook to only update the pieces necessary for alignment. If you have a pre-2.0 index, please run the v2 migration notebook first!

NOTE: we recommend regenerating your settings.yml with the latest version of GraphRAG using `graphrag init`. Copy your LLM settings into it before running this notebook. This ensures your config is aligned with the latest version for the migration.

This notebook will also update your settings.yaml to ensure compatibility with our newer vector store collection naming scheme in order to avoid re-ingesting.

WARNING: This will overwrite your parquet files, you may want to make a backup!

In [7]:
# This is the directory that has your settings.yaml
PROJECT_DIRECTORY = "/Users/naevans/graphrag/working/migration"

In [15]:
from pathlib import Path

from graphrag.config.load_config import load_config
from graphrag.storage.factory import StorageFactory

config = load_config(Path(PROJECT_DIRECTORY))
storage_config = config.output.model_dump()
storage = StorageFactory().create_storage(
    storage_type=storage_config["type"],
    kwargs=storage_config,
)

In [7]:
def remove_columns(df, columns):
    """Remove columns from a DataFrame, suppressing errors."""
    df.drop(labels=columns, axis=1, errors="ignore", inplace=True)

In [8]:
from graphrag.utils.storage import (
    load_table_from_storage,
    write_table_to_storage,
)

text_units = await load_table_from_storage("text_units", storage)

text_units["document_id"] = text_units["document_ids"].apply(lambda ids: ids[0])
remove_columns(text_units, ["document_ids"])

await write_table_to_storage(text_units, "text_units", storage)

## Update settings.yaml
This next section will attempt to insert index names for each vector index using our new schema structure. It depends on most things being default. If you have already customized your vector store schema it may not be necessary.

The primary goal is to align v2 indexes using our old default naming schema with the new customizability. If don't need this done or you have a more complicated config, comment it out and update your config manually to ensure each index name is set.

Old default index names:
- default-text_unit-text
- default-entity-description
- default-community-full_content

v3 versions are:
- text_unit_text
- entity_description
- community_full_content

Therefore, with a v2 index we will explicitly set the old index names so it connects correctly.

NOTE: we are also setting the default vector_size for each index, under the assumption that you are using a prior default with 1536 dimensions. Our new default of text-embedding-3-large has 3072 dimensions, which will be populated as the default if unset. Again, if you have a more complicated situation you may want to manually configure this.


In [ ]:
import yaml

EMBEDDING_DIMENSIONS = 1536

settings = Path(PROJECT_DIRECTORY) / "settings.yaml"
with Path.open(settings) as f:
    conf = yaml.safe_load(f)

vector_store = conf.get("vector_store", {})
container_name = vector_store.get("container_name", "default")
embeddings_schema = vector_store.get("embeddings_schema", {})
text_unit_schema = embeddings_schema.get("text_unit.text", {})
if "index_name" not in text_unit_schema:
    text_unit_schema["index_name"] = f"{container_name}-text_unit-text"
if "vector_size" not in text_unit_schema:
    text_unit_schema["vector_size"] = EMBEDDING_DIMENSIONS
embeddings_schema["text_unit.text"] = text_unit_schema
entity_schema = embeddings_schema.get("entity.description", {})
if "index_name" not in entity_schema:
    entity_schema["index_name"] = f"{container_name}-entity-description"
if "vector_size" not in entity_schema:
    entity_schema["vector_size"] = EMBEDDING_DIMENSIONS
embeddings_schema["entity.description"] = entity_schema
community_schema = embeddings_schema.get("community.full_content", {})
if "index_name" not in community_schema:
    community_schema["index_name"] = f"{container_name}-community-full_content"
if "vector_size" not in community_schema:
    community_schema["vector_size"] = EMBEDDING_DIMENSIONS
embeddings_schema["community.full_content"] = community_schema
vector_store["embeddings_schema"] = embeddings_schema
conf["vector_store"] = vector_store

with Path.open(settings, "w") as f:
    yaml.safe_dump(conf, f)